# Глагольное управление в классах
### Выполнила Елизавета Клыкова, БКЛ-181

**Требования:**
* 2 ✔ – Использование классов: *метакласс Word, классы Verb, Noun, Preposition; класс Combination для сочетаний.*
* 2 ✔ – Использование наследования: *Verb, Noun и Preposition наследуются от Word.*
* 1 ✔ – Три библиотеки из тех, что приходится ставить своими руками: *tqdm, conllu, deeppavlov*
* 2 ✔ "Синтаксический сахар" Питона (декораторы, перегрузка операторов, исключения, примочки из последнего стандарта, ...): *переопределение len() для ConlluReader, исключение ConlluNotLoaded*
* 1 ✔ – Юнит-тесты: *тест для ConlluReader, тест удаления комбинаций*
* 2 **?** – Красивая структура классов, проекта, решения; субъективное мнение преподавателя.
* 1 ✘ – Шаблоны проектирования.
* 1 ✔ – Бонусы: *непосильный труд по записи пар :)*

**Материалы с пар:**
* полезное про [классы](https://github.com/klyshinsky/Object_Oriented_Python_2022_23/blob/main/lecture_20220924_operators_decorators.ipynb)
* [наследование](https://github.com/klyshinsky/Object_Oriented_Python_2022_23/blob/main/lecture_20221008_Inheritance.ipynb)
* [юнит-тесты](https://github.com/klyshinsky/Object_Oriented_Python_2022_23/blob/main/lecture_20221022_unittest.ipynb)
* [шаблоны проектирования](https://github.com/klyshinsky/Object_Oriented_Python_2022_23/blob/main/lecture_20221126_design_patterns.ipynb)

In [1]:
# !pip install tqdm
# !pip install conllu
# !pip install razdel
# !pip install deeppavlov --user
# !python -m deeppavlov install syntax_ru_syntagrus_bert
# !python -m deeppavlov install morpho_ru_syntagrus_pymorphy

In [2]:
import conllu
import random
import unittest
from conllu.exceptions import ParseException
from tqdm.auto import tqdm
from collections import Counter, OrderedDict
from pprint import pprint

import re
import json
import os
from deeppavlov import build_model, configs
from razdel import sentenize, tokenize
from conllu import parse

#### Не перезапускаю в целях экономии времени

In [ ]:
dp_model = build_model('ru_syntagrus_joint_parsing', download=True)

In [ ]:
def analyze_text(text):
    parsed_sentences = []
    if len(text) > 65 and not re.match('https?://', text):
        sentences = [sent.text for sent in list(sentenize(text))]
        for sent in sentences:
            tokens = [token.text for token in list(tokenize(sent))]
            if len(tokens) < 400:
                parsed_sentences.append((sent, parse(dp_model([tokens])[0])[0]))
    return parsed_sentences

In [ ]:
def write_to_conllu(parsed_texts, name):
    with open(name, 'w', encoding = 'utf-8') as f:
        for sent in parsed_texts:
            f.write('# text = ' + sent[0] + '\n')
            f.write(sent[1].serialize())

In [ ]:
def make_conllu(file_in, file_out):
    with open(file_in, 'r', encoding='utf-8') as f:
        for line in tqdm(f.readlines()):
            texts = analyze_text(line)
            if texts:
                write_to_conllu(texts, file_out)

### Считываем разметку в формате .conllu

In [3]:
class ConlluReader:

    def __init__(self, filename, encoding='utf-8'):
        self.filename = filename
        self.encoding = encoding

    def load(self):
        with open(self.filename, 'r', encoding=self.encoding) as f:
            paragraphs = f.read().strip().split('\n\n')
            self.paragraphs = paragraphs

    def parse(self):
        self.load()
        self.tokenlists = []
        # исправлен прошлогодний костыль -- теперь не вылетает ParseException
        for paragraph in tqdm(self.paragraphs):
            parts = paragraph.split('\n1\t')
            clean_par = parts[0].replace('\n', '') + '\n1\t' + parts[1]
            try:
                self.tokenlists.extend([tokenlist for tokenlist
                                        in conllu.parse(clean_par)])
            except ParseException:
                print('The data is not formatted correctly:')
                print(paragraph)
                break
        return self.tokenlists

    # переопределение, исключение
    def __len__(self):
        if hasattr(self, 'paragraphs'):
            return len(self.paragraphs)
        else:
            raise ConlluNotLoadedError

In [4]:
class ConlluNotLoadedError(Exception):
    def __init__(self, message='Call .load() to load the file contents first'):
        self.message = message
        super().__init__(self.message)

### Юнит-тесты

In [5]:
class TestConlluReader(unittest.TestCase):

    def setUp(self):
        print('Setting up ConlluReader...')
        self.conllu = ConlluReader('testfile.CONLLU')

    @classmethod
    def setUpClass(cls):
        print('Setting up the class...')

    def tearDown(self):
        print('Tearing down...')

    def doCleanups(self):
        print('Cleaning up...')

    def testLoad(self):
        print('Testing load function...')
        self.conllu.load()
        self.assertIsInstance(self.conllu.paragraphs, list)
        print('Finished testing load function...')

    def testParse(self):
        print('Testing parse function...')
        self.conllu.parse()
        self.assertIsInstance(self.conllu.tokenlists, list)
        self.assertIsInstance(self.conllu.tokenlists[0],
                              conllu.models.TokenList)
        print('Finished testing parse function...')

    def testData(self):
        print('Testing data completeness...')
        self.conllu.parse()
        self.assertEqual(len(self.conllu.paragraphs),
                         len(self.conllu.tokenlists))
        print('Finished testing data completeness...')

In [6]:
unittest.main(argv=[''], verbosity=2, exit=False)

testData (__main__.TestConlluReader) ... 

Setting up the class...
Setting up ConlluReader...
Testing data completeness...


  0%|          | 0/20823 [00:00<?, ?it/s]

ok
testLoad (__main__.TestConlluReader) ... ok
testParse (__main__.TestConlluReader) ... 

Finished testing data completeness...
Tearing down...
Cleaning up...
Setting up ConlluReader...
Testing load function...
Finished testing load function...
Tearing down...
Cleaning up...
Setting up ConlluReader...
Testing parse function...


  0%|          | 0/20823 [00:00<?, ?it/s]

Finished testing parse function...
Tearing down...
Cleaning up...


ok

----------------------------------------------------------------------
Ran 3 tests in 13.975s

OK


#### Загружаем данные

In [7]:
markup = ConlluReader('articles_Vestnik_rayona.CONLLU')

In [8]:
len(markup)

ConlluNotLoadedError: Call .load() to load the file contents first

In [9]:
markup.load()
len(markup)

20823

In [10]:
tokenlists = markup.parse()

  0%|          | 0/20823 [00:00<?, ?it/s]

### Классы: глаголы, существительные, предлоги

In [11]:
class Word:

    def __init__(self, lemma, pos, form=None, head=None):
        self.lemma = lemma
        self.pos = pos
        self.form = form
        self.head = head

        self.freq = 0
        self.children = Counter()
        self.seen_in = []

In [12]:
class Verb(Word):

    verb_dict = {}

    def __init__(self, lemma, pos='VERB', form=None, head=None):
        super().__init__(lemma, pos, form, head)
        # обновляем список глаголов
        Verb.verb_dict[lemma] = self

    @classmethod
    def getAllVerbs(cls):
        freq_dict = {verb.lemma: verb.freq
                     for verb in list(cls.verb_dict.values())}
        return dict(sorted(freq_dict.items(),
                           key=lambda item: item[1], reverse=True))

    @classmethod
    def delAllVerbs(cls):
        cls.verb_dict = {}

In [13]:
class Noun(Word):

    noun_dict = {}

    def __init__(self, lemma, case, number, animacy, deprel, form,
                 head=None, pos='NOUN'):
        super().__init__(lemma, pos, form, head)
        self.case = case
        self.number = number
        self.animacy = animacy
        self.deprel = deprel
        # обновляем список существительных
        noun_key = lemma + '_' + case + '_' + \
            number + '_' + animacy + '_' + deprel
        Noun.noun_dict[noun_key] = self

In [14]:
class Preposition(Word):

    # предлоги будут уникальны не сами по себе, а в связке с глаголами
    # т.е. бежать-к и обращаться-к -- два разных предлога
    prep_dict = {}

    def __init__(self, lemma, pos='ADP', form=None, head=None):
        super().__init__(lemma, pos, form, head)
        # обновляем список глаголов
        Preposition.prep_dict[lemma] = self

#### Кладем информацию в классы
**Проблема:** диппавлов считает, что предлог зависит от существительного. Мировое лингвистическое сообщество считает наоборот. Будем переворачивать.

In [15]:
def put_words_in_classes(markup):
    # проходим по разборам предложений
    for i, tokens in tqdm(enumerate(markup.tokenlists)):
        sent_id = i  # имитируем уникальное id предложения

        # проходим по токенам предложения
        for token in tokens:

            if token['upos'] == 'VERB':  # нашли глагол
                v_lemma = token['lemma'].lower()
                v_children = []  # сюда положим классы предлогов

                # ищем в токенах зависимые глагола
                for v_child in tokens:
                    if v_child['head'] == token['id']:
                        # глаголы с негативной полярностью
                        if v_child['form'].lower() == 'не':
                            v_lemma = 'не_' + v_lemma
                        # проверяем, что класса для этого глагола еще нет
                        if v_lemma not in Verb.verb_dict:
                            v_as_class = Verb(v_lemma)
                        else:
                            v_as_class = Verb.verb_dict[v_lemma]
                            v_as_class.freq += 1
                            v_as_class.seen_in.append(sent_id)

                        # зависимые существительные, кроме подлежащих (субъектов)
                        if (v_child['upos'] == 'NOUN' or v_child['upos'] == 'PROPN') and v_child['deprel'] != 'nsubj':
                            n_lemma = v_child['lemma'].lower()  # нашли существительное
                            n_form = v_child['form'].lower()
                            try:
                                n_case = v_child['feats']['Case']
                                n_number = v_child['feats']['Number']
                                n_animacy = v_child['feats']['Animacy']
                            except (TypeError, KeyError):
                                n_case, n_number, n_animacy = 'UNK'
                            n_deprel = v_child['deprel']

                            # проверяем, что класса для этого существительного еще нет
                            n_key = '_'.join([n_lemma, n_case, n_number, n_animacy, n_deprel])
                            if n_key not in Noun.noun_dict:
                                n_as_class = Noun(n_lemma, n_case, n_number, n_animacy, n_deprel, n_form)
                            else:
                                n_as_class = Noun.noun_dict[n_key]
                            n_as_class.freq += 1
                            n_as_class.seen_in.append(sent_id)

                            # ищем зависимые предлоги у каждого существительного
                            # если это прямое дополнение, предлога нет
                            p_lemma = '0'
                            p_children = []
                            for n_child in tokens:
                                if n_child['head'] == v_child['id']:
                                    if n_child['upos'] == 'ADP' and n_child['deprel'] == 'case':
                                        p_lemma = n_child['lemma'].lower()  # нашли предлог
                                        # отлавливаем составные предлоги
                                        for p_child in tokens:
                                            try:
                                                if p_child['head'] == n_child['id'] and p_child['deprel'] == 'fixed':
                                                    p_lemma += ' ' + p_child['form'].lower()
                                            except KeyError:
                                                pass

                            p_lemma = v_lemma + '_' + p_lemma
                            # проверяем, что класса для этого предлога еще нет
                            if p_lemma not in Preposition.prep_dict:
                                # родитель предлога -- глагол
                                p_as_class = Preposition(p_lemma, v_as_class)
                            else:
                                p_as_class = Preposition.prep_dict[p_lemma]
                                p_as_class.freq += 1
                                p_as_class.seen_in.append(sent_id)

                            # родитель существительного -- предлог
                            n_as_class.head = p_as_class

                            # предлоги -- дети глаголов, имена -- дети предлогов
                            v_children.append(p_as_class)
                            p_children.append(n_as_class)

                            # обновляем Counter предлога зависимыми-существительными
                            p_as_class.children.update(p_children)
                # обновляем Counter глагола зависимыми-предлогами
                v_as_class.children.update(v_children)

In [16]:
put_words_in_classes(markup)

0it [00:00, ?it/s]

#### Что же получилось?

In [17]:
len(Verb.verb_dict)

4051

In [18]:
Verb.getAllVerbs()

{'стать': 2660,
 'состояться': 1902,
 'пройти': 1579,
 'мочь': 1463,
 'быть': 1266,
 'читать': 1197,
 'принять': 1174,
 'получить': 1080,
 'работать': 1015,
 'проводиться': 1015,
 'провести': 876,
 'проходить': 873,
 'являться': 866,
 'находиться': 837,
 'поступить': 817,
 'зарегистрировать': 792,
 'занять': 756,
 'иметь': 648,
 'произойти': 635,
 'рассказать': 617,
 'жить': 597,
 'отметить': 588,
 'составить': 587,
 'проводить': 583,
 'прийти': 560,
 'представить': 549,
 'составлять': 525,
 'начать': 524,
 'посвятить': 512,
 'установить': 512,
 'решить': 497,
 'ждать': 467,
 'заниматься': 463,
 'идти': 458,
 'появиться': 457,
 'знать': 424,
 'начаться': 424,
 'сделать': 423,
 'говорить': 421,
 'нет': 421,
 'оставаться': 412,
 'принимать': 411,
 'становиться': 408,
 'остаться': 394,
 'вручить': 390,
 'обратиться': 385,
 'совершить': 383,
 'оказаться': 383,
 'отмечать': 379,
 'выступить': 376,
 'смочь': 371,
 'участвовать': 363,
 'организовать': 361,
 'отмечаться': 358,
 'получать': 356

In [19]:
len(Noun.noun_dict)

11839

In [20]:
list(Noun.noun_dict.keys())[-130:-120]

['сказка_Loc_Sing_Inan_advcl',
 'свадьба_Ins_Sing_Inan_obl',
 'откровение_Ins_Plur_Inan_obl',
 'проза_Nom_Sing_Inan_nsubj:pass',
 'условие_Nom_Sing_Inan_conj',
 'письмо_Gen_Plur_Inan_obl',
 'послание_Acc_Sing_Inan_obj',
 'спонсорами—магазина_Ins_Plur_Inan_obl',
 'передача_Acc_Plur_Inan_obl',
 'блюдо_Acc_Plur_Inan_obj']

In [21]:
len(Preposition.prep_dict)

8528

In [22]:
list(Preposition.prep_dict.keys())[-130:-120]

['настоящие—сделать_0',
 'не_настоящие—сделать_из',
 'цель—предоставить_0',
 'цель—предоставить_в',
 'расплатиться_с',
 'грешить_0',
 'удаться_из',
 'возникнуть_по',
 'заменить_в',
 'предостеречь_0']

Откуда-то взялись страшные склеенные леммы. Неужели правда?

![пример](bad_preposition.jpg)

**На будущее:** диппавлов не понимает тире, приклеенное к словам. Нужно чистить или исправлять перед разметкой.

### Сочетания
Мы получили набор классов, связанных между собой. Как из этого получить сочетания? Можно пройтись снизу вверх по построенным связям. Но делать это каждый раз -- как-то странно и неудобно, особенно если есть классы.

In [23]:
class Combination():

    # информация о том, что уже встретилось
    comb_dict = {}

    def __init__(self, verb, prep, noun, text):
        self.verb = verb
        self.prep = prep
        self.deprel = noun.deprel
        self.noun = noun
        self.text = text
        self.freq = 0
        self.seen_in = []
        comb_key = '_'.join([verb.lemma, prep.lemma, noun.case, noun.number,
                             noun.animacy, noun.deprel, noun.lemma])
        Combination.comb_dict[comb_key] = self

    @classmethod
    def getAllCombos(cls):
        all_combos = [value.text for value in list(cls.comb_dict.values())]
        return OrderedDict(Counter(all_combos).most_common())

    @classmethod
    def deleteAllCombos(cls):
        cls.comb_dict = {}

In [24]:
def create_combos():
    for v_lemma, verb in tqdm(list(Verb.verb_dict.items())):
        for prep in verb.children:
            for noun in prep.children:
                comb_key = '_'.join([verb.lemma, prep.lemma.split('_')[-1], noun.case, noun.number, noun.animacy, noun.deprel, noun.lemma])
                if comb_key not in Combination.comb_dict:
                    text = ' '.join([verb.lemma, prep.lemma.split('_')[-1], noun.form])
                    comb = Combination(verb, prep, noun, text)
                    comb.freq += 1
                else:
                    comb = Combination.comb_dict[comb_key]
                    comb.freq += 1

In [25]:
create_combos()

  0%|          | 0/4051 [00:00<?, ?it/s]

In [26]:
Combination.getAllCombos()

OrderedDict([('составить 0 млн', 5),
             ('обучаться 0 сош', 5),
             ('занять 0 место', 4),
             ('не_занять 0 место', 4),
             ('не_зарегистрировать 0 дтп', 4),
             ('отметить 0 день', 3),
             ('набрать 0 воды', 3),
             ('составлять 0 руб', 3),
             ('составлять 0 га', 3),
             ('составить 0 тысяч', 3),
             ('составить 0 руб', 3),
             ('установить 0 лицо', 3),
             ('возбудить 0 дела', 3),
             ('выявить 0 факт', 3),
             ('отмечаться 0 день', 3),
             ('зарегистрировать 0 дтп', 3),
             ('не_иметь 0 детей', 3),
             ('представить 0 команды', 3),
             ('не_представить 0 команды', 3),
             ('заключить 0 договора', 3),
             ('не_установить 0 лицо', 3),
             ('завоевать 0 место', 3),
             ('просиять 0 п', 2),
             ('быть 0 летом', 2),
             ('быть с стороны', 2),
             ('получить 0 свид

А если в дверь стучатся злобные синтаксисты, и мы решили срочно все удалить? Сработает ли наша функция? Проверим это.

### Юнит-тесты 2.0

In [27]:
class TestCombination(unittest.TestCase):

    def testDeleting(cls):
        print('Testing deleteAllCombos()...')
        Combination.comb_dict = {'not empty dict': 'for checking'}
        Combination.deleteAllCombos()
        cls.assertEqual(Combination.comb_dict, {})
        print('Testing done.')

In [28]:
unittest.main(argv=[''], verbosity=2, exit=False)

testDeleting (__main__.TestCombination) ... ok
testData (__main__.TestConlluReader) ... 

Testing deleteAllCombos()...
Testing done.
Setting up the class...
Setting up ConlluReader...
Testing data completeness...


  0%|          | 0/20823 [00:00<?, ?it/s]

ok
testLoad (__main__.TestConlluReader) ... 

Finished testing data completeness...
Tearing down...
Cleaning up...
Setting up ConlluReader...
Testing load function...
Finished testing load function...
Tearing down...
Cleaning up...


ok
testParse (__main__.TestConlluReader) ... 

Setting up ConlluReader...
Testing parse function...


  0%|          | 0/20823 [00:00<?, ?it/s]

Finished testing parse function...
Tearing down...
Cleaning up...


ok

----------------------------------------------------------------------
Ran 4 tests in 14.315s

OK


In [29]:
Combination.deleteAllCombos()

In [30]:
Combination.getAllCombos()

OrderedDict()

### Единый пайплайн -- от файла с текстом до словаря

In [43]:
def create_government_dict(file_in, file_out):
#     размечаем
#     make_conllu(file_in, file_out)

    # считываем разметку
    markup = ConlluReader(file_out)
    markup.parse()
    put_words_in_classes(markup)
    create_combos()
    pprint(Combination.getAllCombos())

In [44]:
create_government_dict('articles_Vestnik_rayona.txt',
                       'articles_Vestnik_rayona.conllu')

  0%|          | 0/20823 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/4051 [00:00<?, ?it/s]

OrderedDict([('составить 0 млн', 5),
             ('обучаться 0 сош', 5),
             ('занять 0 место', 4),
             ('не_занять 0 место', 4),
             ('не_зарегистрировать 0 дтп', 4),
             ('отметить 0 день', 3),
             ('набрать 0 воды', 3),
             ('составлять 0 руб', 3),
             ('составлять 0 га', 3),
             ('составить 0 тысяч', 3),
             ('составить 0 руб', 3),
             ('установить 0 лицо', 3),
             ('возбудить 0 дела', 3),
             ('выявить 0 факт', 3),
             ('отмечаться 0 день', 3),
             ('зарегистрировать 0 дтп', 3),
             ('не_иметь 0 детей', 3),
             ('представить 0 команды', 3),
             ('не_представить 0 команды', 3),
             ('заключить 0 договора', 3),
             ('не_установить 0 лицо', 3),
             ('завоевать 0 место', 3),
             ('просиять 0 п', 2),
             ('быть 0 летом', 2),
             ('быть с стороны', 2),
             ('получить 0 свид

             ('определить 0 территорий', 1),
             ('определить 0 рейтинг', 1),
             ('определить 0 хозяев', 1),
             ('определить по итогам', 1),
             ('определить по прибытию', 1),
             ('определить по счету', 1),
             ('определить по дозвону', 1),
             ('определить по данным', 1),
             ('определить по плану', 1),
             ('определить при жизни', 1),
             ('определить после мероприятия', 1),
             ('определить для выдвижения', 1),
             ('определить к ремонту', 1),
             ('выбирать 0 главу', 1),
             ('выбирать 0 понятие', 1),
             ('выбирать 0 платья', 1),
             ('выбирать 0 делегатов', 1),
             ('выбирать 0 оператора', 1),
             ('выбирать 0 здоровье', 1),
             ('выбирать 0 профессию', 1),
             ('выбирать 0 район', 1),
             ('выбирать 0 песни', 1),
             ('выбирать 0 сад', 1),
             ('выбирать 0 куртку', 1),
   

...

             ('зайти на сайт', 1),
             ('зайти о принципах', 1),
             ('поздравить с окончанием', 1),
             ('поздравить с днем', 1),
             ('поздравить с праздником', 1),
             ('поздравить с юбилеем', 1),
             ('поздравить с 79-летием', 1),
             ('поздравить с девяностолетием', 1),
             ('поздравить с датой', 1),
             ('поздравить с событием—получением', 1),
             ('поздравить с вступлением', 1),
             ('поздравить с событием', 1),
             ('поздравить 0 ребят', 1),
             ('поздравить 0 детей', 1),
             ('поздравить 0 выпускников', 1),
             ('поздравить 0 василия', 1),
             ('поздравить 0 членов', 1),
             ('поздравить 0 друг', 1),
             ('поздравить 0 сотрудников', 1),
             ('поздравить 0 ветерана', 1),
             ('поздравить 0 отца', 1),
             ('поздравить 0 юбиляра', 1),
             ('поздравить 0 тружеников', 1),
             ('

             ('позволить с учетом', 1),
             ('возобновить в году', 1),
             ('возобновить в вторник', 1),
             ('возобновить в деревне', 1),
             ('возобновить 0 расследование', 1),
             ('возобновить 0 работу', 1),
             ('возобновить 0 работа', 1),
             ('возобновить 0 года', 1),
             ('возобновить 0 демонстрации', 1),
             ('возобновить по инициативе', 1),
             ('исполнять 0 обязанности', 1),
             ('исполнять 0 долг', 1),
             ('исполнять 0 композиции', 1),
             ('исполнять 0 песни', 1),
             ('исполнять 0 прошения', 1),
             ('исполнять 0 танцы', 1),
             ('исполнять 0 закон', 1),
             ('исполнять 0 и', 1),
             ('исполнять за пределами', 1),
             ('исполнять по произведению', 1),
             ('исполнять в нарушение', 1),
             ('исполнять в месяцев', 1),
             ('валить горит—из дома', 1),
             ('валить 0 дере

             ('гарантировать 0 клиентам', 1),
             ('гарантировать 0 возможность', 1),
             ('гарантировать в раз', 1),
             ('гарантировать в сроки', 1),
             ('гарантировать с уверенностью', 1),
             ('отстегнуть 0 крепления', 1),
             ('высадить 0 пассажиров', 1),
             ('высадить 0 капуста', 1),
             ('высадить 0 сеянцы', 1),
             ('высадить на площади', 1),
             ('приоткрыть 0 дверцу', 1),
             ('ставить 0 вешку', 1),
             ('ставить 0 раза', 1),
             ('ставить 0 шунты', 1),
             ('ставить 0 оценку', 1),
             ('ставить 0 вопрос', 1),
             ('ставить 0 цели', 1),
             ('ставить 0 знак', 1),
             ('ставить 0 машины', 1),
             ('ставить 0 задачи', 1),
             ('ставить 0 спектакли', 1),
             ('ставить 0 свечи', 1),
             ('ставить 0 диагноз', 1),
             ('ставить 0 уклонистам', 1),
             ('ставить 0 отмет

             ('подняться з года', 1),
             ('подняться на ступень', 1),
             ('подняться на призы', 1),
             ('подняться на пьедестал', 1),
             ('подняться на место', 1),
             ('подняться на этаж', 1),
             ('подняться выше—в класс', 1),
             ('дозвониться до помощи', 1),
             ('дозвониться через номер', 1),
             ('дозвониться по номеру', 1),
             ('не_дождаться 0 ответа', 1),
             ('указывать в телефонах', 1),
             ('указывать в графе', 1),
             ('указывать 0 номера', 1),
             ('указывать 0 служба', 1),
             ('указывать 0 отделение', 1),
             ('указывать 0 номер', 1),
             ('указывать 0 способ', 1),
             ('указывать для зачисления', 1),
             ('указывать на происхождение', 1),
             ('не_удивляться на деле', 1),
             ('ощущать в многом', 1),
             ('ощущать в воображении', 1),
             ('ощущать 0 поддержку', 

             ('дорабатывать 0 стаж', 1),
             ('не_набрать 0 обороты', 1),
             ('посовещаться с семьей', 1),
             ('разглядеть в куклиных', 1),
             ('разглядеть 0 людей', 1),
             ('убедить 0 татьяну', 1),
             ('записывать 0 заявки', 1),
             ('записывать на видео', 1),
             ('привозить 0 товар', 1),
             ('привозить 0 раз', 1),
             ('привозить 0 календари', 1),
             ('привозить по заказу', 1),
             ('привозить на память', 1),
             ('избраннику—вести 0 борьбу', 1),
             ('выдвигать от района', 1),
             ('выдвигать в состав', 1),
             ('заслушиваться 0 отчёты', 1),
             ('накладывать на образование', 1),
             ('не_сдвинуть с места', 1),
             ('не_решить 0 проблему', 1),
             ('не_решить 0 утром', 1),
             ('не_решить 0 судьбу', 1),
             ('не_решить 0 платье', 1),
             ('не_решить 0 детали', 1),
       

И напоследок -- бонус:

![бонус](bogadelnya.jpg)